In [1]:
import larch as lx
import pandas as pd
from larch import P,X

import numpy as np
import xarray as xr
from pytest import approx

/Users/jeffnewman/LocalGit/larix/larix/model/numbamodel.py:23: UserWarning: 

##### larix is experimental, and not feature-complete  #####
the first time you import on a new system, this package will
compile optimized binaries for your machine, which may take 
a little while, please be patient 

  warnings.warn( ### EXPERIMENTAL ### )
OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
raw = pd.read_csv(lx.example_file('swissmetro.csv.gz'))
raw['SM_COST'] = raw['SM_CO'] * (raw["GA"]==0) 
raw['TRAIN_COST'] = raw.eval("TRAIN_CO * (GA == 0)") 
raw['TRAIN_COST_SCALED'] = raw['TRAIN_COST'] / 100
raw['TRAIN_TT_SCALED'] = raw['TRAIN_TT'] / 100

raw['SM_COST_SCALED'] = raw.eval('SM_COST / 100')
raw['SM_TT_SCALED'] = raw['SM_TT'] / 100

raw['CAR_CO_SCALED'] = raw['CAR_CO'] / 100
raw['CAR_TT_SCALED'] = raw['CAR_TT'] / 100
raw['CAR_AV_SP'] = raw.eval("CAR_AV * (SP!=0)")
raw['TRAIN_AV_SP'] = raw.eval("TRAIN_AV * (SP!=0)")

raw['keep'] = raw.eval("PURPOSE in (1,3) and CHOICE != 0")

In [3]:
raw

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_COST,TRAIN_COST_SCALED,TRAIN_TT_SCALED,SM_COST_SCALED,SM_TT_SCALED,CAR_CO_SCALED,CAR_TT_SCALED,CAR_AV_SP,TRAIN_AV_SP,keep
0,2,0,1,1,1,0,1,1,0,3,...,48,0.48,1.12,0.52,0.63,0.65,1.17,1,1,True
1,2,0,1,1,1,0,1,1,0,3,...,48,0.48,1.03,0.49,0.60,0.84,1.17,1,1,True
2,2,0,1,1,1,0,1,1,0,3,...,48,0.48,1.30,0.58,0.67,0.52,1.17,1,1,True
3,2,0,1,1,1,0,1,1,0,3,...,40,0.40,1.03,0.52,0.63,0.52,0.72,1,1,True
4,2,0,1,1,1,0,1,1,0,3,...,36,0.36,1.30,0.42,0.63,0.84,0.90,1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,3,1,1,1192,4,1,7,1,0,5,...,13,0.13,1.48,0.17,0.93,0.56,1.56,1,1,False
10724,3,1,1,1192,4,1,7,1,0,5,...,12,0.12,1.48,0.16,0.96,0.70,0.96,1,1,False
10725,3,1,1,1192,4,1,7,1,0,5,...,16,0.16,1.48,0.16,0.93,0.56,0.96,1,1,False
10726,3,1,1,1192,4,1,7,1,0,5,...,16,0.16,1.78,0.17,0.96,0.91,0.96,1,1,False


In [4]:
data = lx.Dataset.construct.from_idco(raw).dc.query_cases('keep')

In [5]:
m1 = lx.Model(
    data.dc.set_altids([1,2,3])
)
m1.availability_co_vars = {
    1: "TRAIN_AV_SP",
    2: "SM_AV",
    3: "CAR_AV_SP",
}
m1.choice_co_code = 'CHOICE'

m1.utility_co[1] = P("ASC_TRAIN") + X("TRAIN_COST_SCALED") * P("B_COST")
m1.utility_co[2] = X("SM_COST_SCALED") * P("B_COST")
m1.utility_co[3] = P("ASC_CAR") + X("CAR_CO_SCALED") * P("B_COST")

In [6]:
m1.parameters

<xarray.Dataset>
Dimensions:     (param_name: 3)
Coordinates:
  * param_name  (param_name) <U9 'ASC_CAR' 'ASC_TRAIN' 'B_COST'
Data variables:
    value       (param_name) float32 0.0 0.0 0.0
    initvalue   (param_name) float32 0.0 0.0 0.0
    nullvalue   (param_name) float32 0.0 0.0 0.0
    minimum     (param_name) float32 -inf -inf -inf
    maximum     (param_name) float32 inf inf inf
    holdfast    (param_name) int8 0 0 0

In [7]:
m2 = lx.Model(
    data.dc.set_altids([1,2,3])
)
m2.availability_co_vars = {
    1: "TRAIN_AV_SP",
    2: "SM_AV",
    3: "CAR_AV_SP",
}
m2.choice_co_code = 'CHOICE'

m2.utility_co[1] = P("ASC_TRAIN") + X("TRAIN_TT_SCALED") * P("B_TIME") + X("TRAIN_COST_SCALED") * P("B_COST")
m2.utility_co[2] = X("SM_TT_SCALED") * P("B_TIME") + X("SM_COST_SCALED") * P("B_COST")
m2.utility_co[3] = P("ASC_CAR") + X("CAR_TT_SCALED") * P("B_TIME") + X("CAR_CO_SCALED") * P("B_COST")

In [8]:
m3 = lx.Model(
    data.dc.set_altids([1,2,3])
)
m3.availability_co_vars = {
    1: "TRAIN_AV_SP",
    2: "SM_AV",
    3: "CAR_AV_SP",
}
m3.choice_co_code = 'CHOICE'

m3.utility_co[1] = X("TRAIN_COST_SCALED") * P("Z_COST")
m3.utility_co[2] = X("SM_COST_SCALED") * P("Z_COST")
m3.utility_co[3] = X("CAR_CO_SCALED") * P("Z_COST")

m3.groupid = 'ID'

In [9]:
import jax.numpy as jnp

In [10]:
mk = lx.Model(
    data.dc.set_altids([101, 102, 103])
)
mk.utility_co[102] = P("W_OTHER")
mk.utility_co[103] = P("W_COST")

In [32]:
b = lx.LatentClass(
    mk, 
    {101:m1, 102:m2, 103:m3}, 
    datatree=data.dc.set_altids([1,2,3]),
    groupid="ID",
)

In [33]:
b

In [34]:
b.lock(Z_COST=-10000)

In [35]:
b.reflow_data_arrays()

In [36]:
b.groupid

'ID'

In [37]:
b.jax_loglike(b.pvals)

DeviceArray(-7359.497, dtype=float32)

In [45]:
b.jax_loglike_casewise(b.pvals)

DeviceArray([-10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976,
             -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976,
             -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976,
             -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976,
              -9.887511,  -9.887511,  -9.887511,  -9.887511,  -7.286221,  -9.887511,  -9.887511,  -9.887511,
              -7.286221,  -7.286221,  -9.887511,  -7.286221,  -7.286221,  -9.887511,  -7.286221,  -7.286221,
              -7.286221,  -7.286221,  -7.286221,  -9.887511,  -7.286221,  -9.887511, -10.292976, -10.292976,
             -10.292976, -10.292976, -10.292976,  -1.098511, -10.292976, -10.292976, -10.292976, -10.292976,
             -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976, -10.292976,
             -10.29

In [42]:
b.jax_probability(b.pvals)

DeviceArray([[[ 0.555556,  0.222222,  0.222222],
              [ 0.555556,  0.222222,  0.222222],
              [ 0.555556,  0.222222,  0.222222],
              [ 0.555556,  0.222222,  0.222222],
              [ 0.555556,  0.222222,  0.222222],
              [ 0.555556,  0.222222,  0.222222],
              [ 0.555556,  0.222222,  0.222222],
              [ 0.555556,  0.222222,  0.222222],
              [ 0.555556,  0.222222,  0.222222]],

             [[ 0.222222,  0.222222,  0.555556],
              [ 0.222222,  0.222222,  0.555556],
              [ 0.222222,  0.222222,  0.555556],
              [ 0.222222,  0.222222,  0.555556],
              [ 0.222222,  0.222222,  0.555556],
              [ 0.222222,  0.222222,  0.555556],
              [ 0.222222,  0.222222,  0.555556],
              [ 0.222222,  0.222222,  0.555556],
              [ 0.222222,  0.222222,  0.555556]],

             [[ 0.555556,  0.222222,  0.222222],
              [ 0.555556,  0.222222,  0.222222],
              [ 

In [38]:
b.pvals

array([     0.,      0.,      0.,      0.,      0.,      0., -10000.])

In [39]:
assert b.jax_loglike(b.pvals) == approx(-6867.245, rel=1e-4)

AssertionError: 

In [40]:
b.jax_loglike(b.pvals)

DeviceArray(-7359.497, dtype=float32)

In [41]:
b.jax_d_loglike(b.pvals) # 5.25s

2022-03-29 09:18:11.651995: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:55] 
********************************
Slow compile?  XLA was built without compiler optimizations, which can be slow.  Try rebuilding with -c opt.
Compiling module jit_func__26.7823
********************************


DeviceArray([ -441.477  , -1363.4967 ,   135.41975,  -741.098  ,  -169.83261,    84.9163 ,     0.     ], dtype=float32)

In [21]:
b.jax_d_loglike(b.pvals) # 5.25s

DeviceArray([ -441.477  , -1363.4967 ,   135.41975,  -741.098  ,  -169.83261,    84.9163 ,     0.     ], dtype=float32)

In [22]:
result = b.jax_maximize_loglike()

In [23]:
result

     jac: array([ 0.047318,  0.100569,  0.091484, -0.006104, -0.028164,  0.023941,  0.      ])
 loglike: -5317.26904296875
 message: 'Optimization terminated successfully'
    nfev: 42
     nit: 15
    njev: 15
  status: 0
 success: True
       x: array([-5.446250e-01, -1.333806e+00, -3.895746e-01, -3.286683e+00, -1.520202e+00, -5.502715e-02, -1.000000e+04])

In [ ]:
assert result.loglike == approx(-4474.478515625, rel=1e-5)

In [ ]:
result.loglike

In [ ]:
b.parameters.to_dataframe()

In [ ]:
se, hess, ihess = b.jax_param_cov(b.pvals)

In [ ]:
b.parameters.to_dataframe()

In [ ]:
assert b.pstderr == approx(np.array([ 
    0.048158,  0.069796,  0.069555,  0.106282,  0.161079,  0.11945 ,  0.
]), rel=5e-3)

In [ ]:
b.pstderr

In [ ]:
assert b.pvals == approx(np.array([ 
    6.079781e-02, -9.362056e-01, -1.159657e+00, -3.095285e+00, -7.734768e-01,  1.155985e+00, -1.000000e+04
]), rel=5e-3)

In [ ]:
b.pvals

In [ ]:
ds = xr.Dataset()
ds = ds.dc.set_altids([1,2,3,4])
ds.dc.altids()

In [ ]:
ds.dc.n_alts

In [ ]:
ds = ds.dc.set_altids([7,8,9], dim_name='A')


In [ ]:
ds.dc.altids()